In [66]:
%load_ext autoreload
%autoreload 2
import os
from dotenv import load_dotenv
load_dotenv("/Users/gkreder/bioimageio-chatbot/.vscode/.env")
from bioimageio_chatbot.chatbot import create_customer_service, QuestionWithHistory, UserProfile
from schema_agents.schema import Message
import asyncio
import yaml
import pytest
from tqdm.auto import tqdm
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from bioimageio_chatbot.knowledge_base import create_vector_knowledge_base
from bioimageio_chatbot.chatbot import load_model_info
from bioimageio_chatbot.image_processor import ImageProcessor
import requests
import re
import sys
import numpy as np

In [20]:
from bioimageio_chatbot.utils import get_manifest, download_file

In [109]:

models = load_model_info()
db_path = '/Users/gkreder/Downloads/image_db'
yaml_dir = os.path.join(db_path, 'rdf_sources')
input_images_dir = os.path.join(db_path, "input_images")
embeddings_dir = os.path.join(db_path, "embeddings")
for d in [yaml_dir, input_images_dir, embeddings_dir]:
    os.makedirs(d, exist_ok=True)

all_metadata = []
all_embeddings = []
image_embedder = ImageProcessor()
for i_m, m in enumerate(tqdm(models)):
    response = requests.get(m['rdf_source'])
    if response.status_code == 200:
        rdf_source_content = response.content.decode('utf-8')
        rdf_dict = yaml.safe_load(rdf_source_content)
        try:
            nickname = rdf_dict['config']['bioimageio']['nickname']
            test_inputs = rdf_dict['test_inputs']
            input_axes = rdf_dict['inputs'][0]['axes']
        except:
            continue
        with open(os.path.join(yaml_dir, f"{nickname}.yaml"), 'w') as f:
            f.write(rdf_source_content)
        mislabeled_axes = {'impartial-shark' : 'yx'}
        if nickname in mislabeled_axes:
            input_axes = mislabeled_axes[nickname]
        for i_ti, ti in enumerate(test_inputs):
            if "input_test_time_for" in ti: # nice-peacock
                continue
            response = requests.get(ti)
            if response.status_code == 200:
                input_image_file = os.path.join(input_images_dir, f"{nickname}.npy")
                with open(input_image_file, 'wb') as file:
                    file.write(response.content)
            embedding = image_embedder.embed_image(input_image_file, input_axes)
            np.save(os.path.join(embeddings_dir, f"{nickname}.npy"), embedding)
            all_metadata.append(rdf_dict)
            all_embeddings.append(embedding)

100%|██████████| 141/141 [02:57<00:00,  1.26s/it]


In [111]:
from langchain.vectorstores import FAISS

In [115]:
all_embedding_pairs = list(zip(all_embeddings, [x['config']['bioimageio']['nickname'] for x in all_metadata]))

In [126]:
all_nicknames = [x['config']['bioimageio']['nickname'] for x in all_metadata]

In [127]:
# vectordb = FAISS.from_embeddings(all_embedding_pairs, all_embeddings, metadatas=all_metadata)
# FAISS.from_embeddings(embedding = all_embeddings, metadatas=all_metadata)
vectordb = FAISS.from_documents(all_nicknames, all_embeddings)
# db = FAISS.from_documents(all_metadata, all_embeddings)

AttributeError: 'str' object has no attribute 'page_content'

In [125]:
import langchain
langchain.__version__

'0.0.335'

In [118]:
set([x.shape for x in all_embeddings])

{(1, 4096)}

In [104]:
vectordb.save_local(os.path.join(db_path, 'model-zoo-images.faiss'), index_name="model-zoo-images")

['affable-shark',
 'impartial-shrimp',
 'hiding-tiger',
 'kind-seashell',
 'chatty-frog',
 'organized-badger',
 'willing-hedgehog',
 'discreet-rooster',
 'humorous-owl',
 'loyal-parrot',
 'fearless-crab',
 'non-judgemental-eagle',
 'emotional-cricket',
 'powerful-chipmunk',
 'shivering-raccoon',
 'creative-panda',
 'conscientious-seashell',
 'powerful-fish',
 'hiding-blowfish',
 'loyal-squid',
 'pioneering-rhino',
 'wild-whale',
 'laid-back-lobster',
 'passionate-t-rex',
 'impartial-shark',
 'thoughtful-turtle',
 'straightforward-crocodile',
 'independent-shrimp',
 'polite-pig',
 'placid-llama',
 'naked-microbe',
 'determined-chipmunk',
 'joyful-deer',
 'nice-peacock',
 'easy-going-sauropod',
 'noisy-fish',
 'ambitious-sloth',
 'organized-cricket',
 'noisy-hedgehog',
 'amiable-crocodile',
 'modest-octopus',
 'ambitious-ant',
 'efficient-chipmunk',
 'courteous-otter']

In [277]:
for m in load_model_info():
    break

In [279]:
m['nickname']

'affable-shark'

In [129]:
# from langchain_core.embeddings import Embeddings
import langchain

In [131]:
from langchain.embeddings.

ImportError: cannot import name 'Embeddings' from 'langchain.embeddings' (/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/langchain/embeddings/__init__.py)

In [132]:
import pickle as pkl

In [134]:
zhang = pkl.load(open("/Users/gkreder/Downloads/vgg16_zhang_perceptual.pkl", 'rb'))

ModuleNotFoundError: No module named 'dnnlib'

In [135]:
import lpips
loss_fn_vgg = lpips.LPIPS(net='vgg')

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/gkreder/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:48<00:00, 11.5MB/s] 


Loading model from: /Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/lpips/weights/v0.1/vgg.pth


In [140]:
t1 = torch.from_numpy(all_embeddings[0])
t2 = torch.from_numpy(all_embeddings[1])
loss_fn_vgg(t1, t2)

RuntimeError: Given input size: (64x1x4096). Calculated output size: (64x0x2048). Output size is too small

In [142]:
t1 = torch.from_numpy(all_embeddings[0])
t2 = torch.from_numpy(all_embeddings[1])

In [143]:
from scipy.spatial.distance import cosine

In [145]:
cosine(t1.flatten(), t2.flatten())

0.6623162031173706

In [146]:
import lpips
import torch
loss_fn_vgg = lpips.LPIPS(net='vgg')

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/lpips/weights/v0.1/vgg.pth


In [244]:
from keras.applications.vgg16 import preprocess_input

resized_images = []
for m in tqdm(all_metadata):
    nickname = m['config']['bioimageio']['nickname']
    input_axes = m['inputs'][0]['axes']
    input_image_path = os.path.join(input_images_dir, f"{nickname}.npy")
    mislabeled_axes = {'impartial-shark' : 'yx'}
    if nickname in mislabeled_axes:
        input_axes = mislabeled_axes[nickname]
    resized_image = image_embedder.resize_image(input_image_path, input_axes)
    transposed_image = np.transpose(resized_image, (0,3,1,2))
    preprocessed_image = preprocess_input(transposed_image)
    torch_image = torch.from_numpy(preprocessed_image.copy()).to(torch.float32)
    resized_images.append(torch_image)

100%|██████████| 44/44 [00:00<00:00, 595.96it/s]


In [281]:
def get_similarity(img1 : torch.Tensor, img2 : torch.Tensor) -> float:
    diff = loss_fn_vgg(img1, img2)
    diff = diff.detach().numpy()[0][0][0][0]
    sim = 1 - diff # Closer to 1 = more similar
    return(sim)

In [290]:
i1 = 2
i2 = 7
sim = loss_fn_vgg(resized_images[i1], resized_images[i2]).detach().numpy()[0][0][0][0]
1 - sim

0.6516184508800507

In [271]:
cv2.imread("/Users/gkreder/Downloads/microscopy-fruit-fly-neurosciencenews.jpeg").shape

(800, 1400, 3)

In [273]:
import scipy
import PIL as pillow
import cv2

image_embedder = ImageProcessor()
user_img_path = "/Users/gkreder/Downloads/microscopy-fruit-fly-neurosciencenews.jpeg"
user_img = cv2.imread(user_img_path)
user_img_axes = "cyx"
user_image_resized = image_embedder.resize_image(user_img, user_img_axes, output_format = "bcyx") # https://github.com/richzhang/PerceptualSimilarity
user_torch = torch.from_numpy(preprocess_input(user_image_resized).copy()).to(torch.float32)



In [274]:
def get_sim(img1, img2):
    s = loss_fn_vgg(img1, img2)
    return(s.detach().numpy()[0][0][0][0])

sims = [get_sim(user_torch, x) for x in tqdm(resized_images)]

100%|██████████| 44/44 [00:05<00:00,  8.42it/s]


In [275]:
all_metadata[np.argmin(sims)]

{'attachments': {'files': []},
 'authors': [{'name': 'A. Wolny'}, {'name': 'V. Bondarenko'}],
 'cite': [{'text': 'V. Bondarenko et al. Ex vivo Engineering Uterine Environment for Peri-implantation Mouse Development',
   'url': 'https://github.com/kreshuklab/mouse-embryo-seg'}],
 'config': {'bioimageio': {'created': '2023-03-27 12:11:52.357154',
   'doi': '10.5281/zenodo.7774490',
   'nickname': 'powerful-fish',
   'nickname_icon': '🐟',
   'owners': [66700],
   'status': 'accepted',
   'version_id': '7774490',
   'version_name': 'version 4'},
  'deepimagej': {'allow_tiling': True,
   'model_keys': None,
   'prediction': {'postprocess': [{'spec': None}],
    'preprocess': [{'spec': None}]},
   'pyramidal_model': False,
   'test_information': {'inputs': [{'name': 'test_input.npy',
      'pixel_size': {'x': 1.0, 'y': 1.0, 'z': 1.0},
      'size': '128 x 128 x 32 x 1'}],
    'memory_peak': None,
    'outputs': [{'name': 'test_output.npy',
      'size': '128 x 128 x 32 x 1',
      'type': 'i

In [293]:
cv2.imread("/USers/gkreder/Downloads/nuclear_blue_image.jpg").shape

(730, 1000, 3)